In [1]:
import pandas as pd
import numpy as np
import os


os.getcwd()

'c:\\Users\\fabri\\OneDrive\\Documentos\\Estudos\\ANAC'

# Functions

In [2]:
def read_tickets():
    archives = os.listdir("Raw")

    final_df = []
    

    for count in archives:
        local_df = pd.read_csv(f'Raw/{count}',sep=';',encoding = 'ISO-8859-1',usecols=lambda column: column not in ['Unnamed: 0'])
        if local_df.shape[1] == 1:
            local_df = pd.read_csv(f'Raw/{count}',sep=',',encoding = 'ISO-8859-1',usecols=lambda column: column not in ['Unnamed: 0'])

        local_df = pd.DataFrame(local_df)
        local_df = local_df.set_axis(['year','month','company','origin','destiny','price','ticket_amount'],axis=1)    

        if count == archives[0]:
            final_df = local_df
        else:
            final_df = pd.concat([final_df,local_df])
    return final_df


def read_auxiliary():

    public_airports = pd.read_csv("Auxiliar/AerodromosPublicos_ajustado.csv",
                              encoding = 'ISO-8859-1',
                              header = 1,
                              sep=';'
                              )
    
    companies = pd.read_csv("Auxiliar/pda_empresas_aereas_nacionais.csv",sep=';',header=1)

    companies = companies[companies['ICAO'].notna()]

    return public_airports,companies


def merging_auxiliary(tickets,public_airports,companies):
    
    df_merged = pd.merge(tickets,
            companies[['ICAO','Razao','Servico']],
            how='left',
            left_on = 'company',
            right_on = 'ICAO')
    
    df_merged = df_merged.drop(columns='ICAO')
       
    df_merged = df_merged.rename(columns={'Razao':'company_name','Servico':'service'})

    # the company IPM wasn't found in auxiliary datasets, so I'm calling company name as IPM too
    df_merged['company_name'] = np.where(df_merged['company'] == 'IPM',
                                         'IPM',df_merged['company_name'])
    
    df_merged['service'] = np.where(df_merged['company'] == 'IPM',
                                         'unknown',df_merged['service'])

    origin = public_airports[['Código OACI','Município','UF']]
    origin = origin.set_axis(['origin','origin_city','origin_state'],axis=1)

    df_merged = pd.merge(df_merged,
                            origin,
                            how='left',
                            on = 'origin')

    destiny = public_airports[['Código OACI','Município','UF']]
    destiny = origin.set_axis(['destiny','destiny_city','destiny_state'],axis=1)

    df_merged = pd.merge(df_merged,
                            destiny,
                            how='left',
                            on = 'destiny')
    

    
    return df_merged


def aggregating_transactions_to_total(df,grp_columns):

    df = df.groupby(grp_columns).agg(
        qty_tickets = ('ticket_amount','sum')
    ).reset_index()

    return df




## Bloco 1 - Carga de dado 
#### Adição de colunas auxiliares como deparas e criação de arquivo de trabalho

In [3]:
tickets = read_tickets()

public_airports,companies = read_auxiliary()

merged_tickets = merging_auxiliary(tickets=tickets,public_airports=public_airports,companies=companies)


In [16]:

group_columns = ['year','month','company','origin','destiny','company_name','service','origin_city','origin_state','destiny_city','destiny_state'] 

tickets_qty = aggregating_transactions_to_total(df = merged_tickets,grp_columns=group_columns)

# merged_tickets



In [4]:
merged_tickets

,year,month,company,origin,destiny,price,ticket_amount,company_name,service,origin_city,origin_state,destiny_city,destiny_state
0,2021,1,AZU,SBAC,SBBR,"293,20",1,AZUL LINHAS AÉREAS BRASILEIRAS S/A,"TRANSPORTE AÉREO NÃO REGULAR, TRANSPORTE AÉREO...",ARACATI,Ceará,BRASÍLIA,Distrito Federal
1,2021,1,AZU,SBAC,SBBR,"340,90",6,AZUL LINHAS AÉREAS BRASILEIRAS S/A,"TRANSPORTE AÉREO NÃO REGULAR, TRANSPORTE AÉREO...",ARACATI,Ceará,BRASÍLIA,Distrito Federal
2,2021,1,AZU,SBAC,SBBR,"367,30",1,AZUL LINHAS AÉREAS BRASILEIRAS S/A,"TRANSPORTE AÉREO NÃO REGULAR, TRANSPORTE AÉREO...",ARACATI,Ceará,BRASÍLIA,Distrito Federal
3,2021,1,AZU,SBAC,SBBR,"435,90",1,AZUL LINHAS AÉREAS BRASILEIRAS S/A,"TRANSPORTE AÉREO NÃO REGULAR, TRANSPORTE AÉREO...",ARACATI,Ceará,BRASÍLIA,Distrito Federal
4,2021,1,AZU,SBAC,SBBR,"452,90",1,AZUL LINHAS AÉREAS BRASILEIRAS S/A,"TRANSPORTE AÉREO NÃO REGULAR, TRANSPORTE AÉREO...",ARACATI,Ceará,BRASÍLIA,Distrito Federal
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21685352,2024,11,TAM,SWLC,SBCT,1367.86,2,TAM LINHAS AÉREAS S.A.,TRANSPORTE AÉREO REGULAR,RIO VERDE,Goiás,CURITIBA,Paraná
21685353,2024,11,TAM,SWLC,SBCT,2288.56,1,TAM LINHAS AÉREAS S.A.,TRANSPORTE AÉREO REGULAR,RIO VERDE,Goiás,CURITIBA,Paraná
21685354,2024,11,TAM,SWLC,SBGL,2778.11,5,TAM LINHAS AÉREAS S.A.,TRANSPORTE AÉREO REGULAR,RIO VERDE,Goiás,RIO DE JANEIRO,Rio de Janeiro
21685355,2024,11,TAM,SWLC,SBGL,2855.38,2,TAM LINHAS AÉREAS S.A.,TRANSPORTE AÉREO REGULAR,RIO VERDE,Goiás,RIO DE JANEIRO,Rio de Janeiro


In [5]:
# Check for null values
merged_tickets.isnull().sum()
# so, there is some null values in the auxiliary columns



year             0
month            0
company          0
origin           0
destiny          0
price            0
ticket_amount    0
company_name     0
service          0
origin_city      0
origin_state     0
destiny_city     1
destiny_state    1
dtype: int64

In [6]:
# In the company name column, we find that the IPM company wasn't found 
# (merged_tickets[merged_tickets['origin_city'].isna()])['company'].unique()

(merged_tickets[merged_tickets['destiny_city'].isna()])['destiny'].unique()

array(['SSPB'], dtype=object)

### Exploratory Data Analysis